In [195]:
function matrixize(lines)
    get_row(line) = split(line, "") .|> x -> parse(Int, x)
    rows = get_row.(lines)
    return hcat(rows...)'
end

matrixize (generic function with 1 method)

In [218]:
lines = readlines("input.txt")
grid = matrixize(lines)

10×10 adjoint(::Matrix{Int64}) with eltype Int64:
 7  6  1  2  6  4  8  2  1  7
 7  6  1  7  2  3  7  6  7  2
 2  8  5  3  8  7  1  8  3  6
 7  2  1  4  3  6  7  1  3  5
 1  5  3  3  3  6  5  6  1  4
 6  2  5  8  1  7  2  8  6  2
 5  3  7  7  6  7  5  5  8  3
 5  6  1  3  2  6  8  2  7  8
 8  3  8  1  1  3  4  4  6  5
 3  4  4  5  4  2  8  7  3  3

In [219]:
function is_in_bounds(point, grid_size)
    return (1 ≤ point[1] ≤ grid_size[1]) && (1 ≤ point[2] ≤ grid_size[2])
end

function get_neighbours(point, grid_size)
    neighbours = CartesianIndex[]
    for dr in [-1, 0, 1]
        for dc in [-1, 0, 1]
            neigh = Tuple(point) .+ (dr, dc)
            if is_in_bounds(neigh, grid_size) && neigh != point
                push!(neighbours, CartesianIndex(neigh))
            end
        end
    end
    return neighbours
end

get_neighbours (generic function with 1 method)

In [220]:
get_flashed_cells(grid, old_flashed) = filter(c -> grid[c] > 9 && c ∉ old_flashed, CartesianIndices(grid))

function propagate_flash(grid, new_flashed, old_flashed)
    grid_size = size(grid)
    neighbours_list = [get_neighbours(c, grid_size) for c in new_flashed if c ∉ old_flashed]
    for neighbours in neighbours_list
        for n in neighbours
            grid[n] += 1
        end
    end
    return grid
end

reset_energy(cell) = cell > 9 ? 0 : cell

function process_step(grid)
    grid = grid .+ 1
    old_flashed = []
    new_flashed = get_flashed_cells(grid, old_flashed)
    while length(new_flashed) > 0
        grid = propagate_flash(grid, new_flashed, old_flashed)
        append!(old_flashed, new_flashed)
        new_flashed = get_flashed_cells(grid, old_flashed)
    end
    grid = reset_energy.(grid)
    return grid, length(old_flashed)
end

function part1(grid)
    number_of_steps = 100
    total_flash = 0
    for s in 1:number_of_steps
        grid, flash_count = process_step(grid)
        total_flash += flash_count
    end
    return total_flash
end

part1 (generic function with 1 method)

In [221]:
part1(grid)

1739

In [222]:
function part2(grid)
    step = 0
    while sum(grid .== 0) != length(grid)
        step += 1
        grid, _ = process_step(grid)
    end
    return step
end

part2 (generic function with 1 method)

In [223]:
part2(grid)

324